<a href="https://colab.research.google.com/github/Wasifnasim/mini_project_1/blob/main/final_Hate_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate accelerate torch scikit-learn pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving twitter(1).csv to twitter(1).csv


In [3]:
import re, json, pandas as pd
from sklearn.model_selection import train_test_split

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df = pd.read_csv("twitter(1).csv")
df['text'] = df['tweet'].astype(str).apply(clean_text)

# map class → labels
label_map = {"0": "Hate Speech", "1": "Offensive Language", "2": "No Hate and Offensive"}
df['label'] = df['class'].astype(int)

with open("label_map.json", "w") as f:
    json.dump(label_map, f, indent=2)

train_df, test_df = train_test_split(df[['text','label']], test_size=0.2, stratify=df['label'], random_state=42)
train_df.to_csv("train_split.csv", index=False)
test_df.to_csv("test_split.csv", index=False)

print("Prepared data → train_split.csv, test_split.csv")


Prepared data → train_split.csv, test_split.csv


In [4]:
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

MODEL_NAME = "distilbert-base-uncased"
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3

# Load label map
import json
with open("label_map.json","r") as f:
    raw_map = json.load(f)
id2label = {int(k): v for k,v in raw_map.items()}
label2id = {v: int(k) for k,v in id2label.items()}

# Load splits
train_df = pd.read_csv("train_split.csv")
test_df  = pd.read_csv("test_split.csv")

train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

train_ds = train_ds.rename_column("label", "labels")
test_ds  = test_ds.rename_column("label", "labels")

train_ds.set_format("torch")
test_ds.set_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE*2,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()

trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

print("✅ Model saved in ./saved_model")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wasifullah2023 (wasifullah2023-acet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.267400,0.249666,0.911438,0.753588
2,0.212800,0.241930,0.917289,0.745659
3,0.191200,0.252006,0.919911,0.771655


✅ Model saved in ./saved_model


In [5]:
from transformers import pipeline
pipe = pipeline("text-classification", model="./saved_model", tokenizer="./saved_model", return_all_scores=True)

examples = [
    "I want to kill them all",
    "Have a nice day, friend",
    "You idiot, shut up"
]

for ex in examples:
    preds = pipe(ex)[0]
    print(ex)
    for p in preds:
        print(f"  {p['label']}: {p['score']:.4f}")
    print()


Device set to use cuda:0


I want to kill them all
  Hate Speech: 0.7441
  Offensive Language: 0.2311
  No Hate and Offensive: 0.0248

Have a nice day, friend
  Hate Speech: 0.0034
  Offensive Language: 0.0127
  No Hate and Offensive: 0.9839

You idiot, shut up
  Hate Speech: 0.0786
  Offensive Language: 0.9094
  No Hate and Offensive: 0.0120



/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
from google.colab import files
!zip -r saved_model.zip saved_model
files.download("saved_model.zip")


  adding: saved_model/ (stored 0%)
  adding: saved_model/config.json (deflated 49%)
  adding: saved_model/vocab.txt (deflated 53%)
  adding: saved_model/tokenizer_config.json (deflated 75%)
  adding: saved_model/special_tokens_map.json (deflated 42%)
  adding: saved_model/tokenizer.json (deflated 71%)
  adding: saved_model/model.safetensors (deflated 8%)
  adding: saved_model/training_args.bin (deflated 54%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>